In [1]:
%matplotlib inline
import xarray as xr
import numpy as np 
import os 

import intake
import zarr 
import gcsfs
from rechunker import rechunk

In [2]:
import dask
import dask.array as da
import dask.distributed as dd
import rhg_compute_tools.kubernetes as rhgk

In [4]:
client, cluster = rhgk.get_standard_cluster()

In [5]:
cluster

pull in some CMIP6 data

In [8]:
# search the catalog
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")
cat = col.search(activity_id='ScenarioMIP', experiment_id='ssp370', table_id='day', variable_id='tasmax')

In [9]:
# access the data and do some cleanup
ds_model = cat['ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp370.day.gr1'].to_dask(
    ).isel(member_id=0).squeeze(drop=True).drop(['height', 'member_id'])

ds_model.lon.values[ds_model.lon.values > 180] -= 360
ds_model = ds_model.roll(lon=72, roll_coords=True)

In [10]:
fs = gcsfs.GCSFileSystem(token='/opt/gcsfuse_tokens/impactlab-data.json')
store_filename = 'gs://impactlab-data/climate/source_data/GFDL-ESM4.ssp370.zarr'
store = fs.get_mapper(store_filename, check=False)

if not fs.exists(store_filename): 
    # save as a zarr store for rechunking 
    ds_model.to_zarr(store, consolidated=True, mode="w")

In [ ]:
# load CMIP6 zarr store, this opens it as a dataset for inspection (note: MUST use path from above)
ds_reloaded = xr.open_zarr(store)

In [11]:
# load CMIP6 zarr store as a zarr group, this is the version that rechunker needs 
source_group = zarr.open_consolidated(store, mode='r')
print(source_group.tree())

/
 ├── lat (180,) float64
 ├── lon (288,) float64
 ├── tasmax (31390, 180, 288) float32
 └── time (31390,) int64


In [12]:
source_array = source_group['tasmax']

use `rechunker` package to rechunk CMIP6 model output from time chunks to space chunks 

In [14]:
target_chunks = {'time': 1, 'lat': 30, 'lon': 30}

# unsure why, but specifying as a string was not working
max_mem = 87091200

tmp_storename = 'gs://impactlab-data/climate/source_data/GFDL-ESM4.ssp370_tmpchunks.zarr'
target_storename = 'gs://impactlab-data/climate/source_data/GFDL-ESM4.ssp370_rechunk.zarr'

temp_store = fs.get_mapper(tmp_storename, create=True)
target_store = fs.get_mapper(target_storename, create=True)

array_plan = rechunk(source_array, target_chunks, max_mem, target_store, temp_store=temp_store)
array_plan

<Rechunked>
* Source      : <zarr.core.Array '/tasmax' (31390, 180, 288) float32 read-only>

* Target      : <zarr.core.Array (31390, 180, 288) float32>

In [15]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    result = array_plan.execute()

check target array to see if the output got rechunked 

In [22]:
# open target array and double check that chunks are what we specified above
source_array_rechunked = zarr.open(target_store, mode='r')
print(source_array_rechunked)

<zarr.core.Array (31390, 180, 288) float32 read-only>


In [23]:
# check chunks
source_array_rechunked.chunks

(1, 30, 30)